In [ ]:
import os
import pandas as pd
import mysql.connector
from tqdm import tqdm

# MySQL connection config
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="B@ndung40175",
    database="epigenomic"
)
cursor = db.cursor()

# Folder with CSV files
csv_folder = "E:/DataTesis/Final_Epigenomic"  # update if needed

# Track processed files
processed_files_path = "processed_files.txt"
if os.path.exists(processed_files_path):
    with open(processed_files_path, "r") as f:
        already_processed = set(f.read().splitlines())
else:
    already_processed = set()

# Get only unprocessed CSVs
csv_files = [
    f for f in os.listdir(csv_folder)
    if f.endswith(".csv") and f not in already_processed
]

for file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(file_path)

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Inserting rows from {file}", leave=False):

        # EXAMPLE: Replace this block with your actual logic based on the ER diagram
        chrom = row['chrom']
        start = int(row['start'])
        end = int(row['end'])
        strand = row['strand']
        tpm = float(row['TPM'])
        feature_name = row['feature_name']
        feature_class = row['feature_class']
        fold_change = float(row['fold_change'])
        window_size = int(row['window_size'])
        region_type = row['region_type']
        cell_line = row['cell_line']

        # You need to handle INSERTS like:
        # - INSERT INTO GenomicRegion
        # - INSERT INTO EpigenomicFeature
        # - INSERT INTO EpigenomicActivity
        # - INSERT INTO CellLine, WindowSize, etc.
        # Make sure to deduplicate if needed (check before inserting)

        # Example: Insert CellLine
        cursor.execute("SELECT cell_line_id FROM CellLine WHERE name = %s", (cell_line,))
        result = cursor.fetchone()
        if result:
            cell_line_id = result[0]
        else:
            cursor.execute("INSERT INTO CellLine (name) VALUES (%s)", (cell_line,))
            db.commit()
            cell_line_id = cursor.lastrowid

        # Repeat for RegionType, WindowSize, GenomicRegion, etc.
        # Then build relationships in EpigenomicActivity and FeatureValue

    # Add to processed list
    with open(processed_files_path, "a") as f:
        f.write(file + "\n")

    print(f"✅ Finished {file}")

cursor.close()
db.close()
print("✅ All files processed.")


In [1]:
import os
import math
import pandas as pd
import mysql.connector
from tqdm import tqdm

def sanitize_value(val):
    if val is None:
        return None
    if isinstance(val, float) and math.isnan(val):
        return None
    return val

# Feature class map
feature_class_map = {
    "Histone Modification": [
        "H3K27ac", "H3K27me3", "H3K36me3", "H3K4me1", "H3K4me2", "H3K4me3",
        "H3K79me2", "H3K9ac", "H3K9me2", "H3K9me3", "H4K20me1"
    ],
    "DNA Methylation": ["DNMT1", "DNMT3B"],
    "Chromatin Remodeling and Histone Modifiers": [
        "EZH2", "EED", "SUZ12", "KDM1A", "KDM2A", "KDM3A", "KDM4B", "KDM5A",
        "KDM5B", "KDM6A", "KAT2A", "KAT2B", "KAT7", "KAT8", "HDAC1", "HDAC2",
        "HDAC3", "HDAC6", "HDAC8", "BRD4"
    ],
    "Chromatin Structures and Remodelling Factors": [
        "CHD1", "CHD2", "CHD4", "CHD7", "ARID1B", "ARID2", "ARID3A", "ARID3B",
        "ARID4A", "ARID4B", "ARID5B", "SMARCA4", "SMARCB1"
    ],
    "Transcription Factor with Epigenetic Roles": [
        "CTCF", "CTCFL", "FOXA1", "FOXA2", "FOXA3", "GATA1", "GATA2", "GATA3",
        "NR3C1", "ESR1", "ETS1", "ETS2"
    ]
}
feature_to_class = {f: cls for cls, features in feature_class_map.items() for f in features}

# MySQL connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="B@ndung40175",
    database="epidb"
)
cursor = db.cursor()

csv_folder = "E:/DataTesis/Final_Epigenomic"
processed_files_path = "processed_files.txt"
already_processed = set()
if os.path.exists(processed_files_path):
    with open(processed_files_path, "r") as f:
        already_processed = set(f.read().splitlines())

csv_files = [
    f for f in os.listdir(csv_folder)
    if f.endswith(".csv") and f not in already_processed
]

# Caches
cell_line_cache = {}
region_type_cache = {}
window_size_cache = {}
feature_ids_cache_global = {}
region_cache = {}
activity_cache = {}

# Correct ID column names
id_column_map = {
    "CellLine": "cell_line_id",
    "RegionType": "region_type_id",
    "WindowSize": "window_size_id"
}

def get_or_insert_id(table, column, value, cache):
    if value in cache:
        return cache[value]

    id_column = id_column_map[table]
    cursor.execute(f"SELECT {id_column} FROM {table} WHERE {column} = %s", (value,))
    res = cursor.fetchone()
    
    if res:
        cache[value] = res[0]
        return res[0]

    cursor.execute(f"INSERT INTO {table} ({column}) VALUES (%s)", (value,))
    db.commit()  # ✅ Fix: use the correct connection
    new_id = cursor.lastrowid
    cache[value] = new_id
    return new_id

for file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(file_path)

    base_name = os.path.basename(file).replace("_epigenomic.csv", "")
    cell_line, window_size, region_type = base_name.split("_")
    window_size = int(window_size)

    cell_line_id = get_or_insert_id("CellLine", "name", cell_line, cell_line_cache)
    region_type_id = get_or_insert_id("RegionType", "type", region_type, region_type_cache)
    window_size_id = get_or_insert_id("WindowSize", "size", window_size, window_size_cache)

    fixed_cols = {"chrom", "start", "end", "strand", "TPM"}
    feature_cols = [col for col in df.columns if col not in fixed_cols]

    for feature_name in feature_cols:
        if feature_name in feature_ids_cache_global:
            continue
        feature_class = feature_to_class.get(feature_name, "Unknown")
        cursor.execute(
            "SELECT feature_id FROM EpigenomicFeature WHERE name = %s AND feature_class = %s",
            (feature_name, feature_class)
        )
        res = cursor.fetchone()
        if res:
            feature_ids_cache_global[feature_name] = res[0]
        else:
            cursor.execute(
                "INSERT INTO EpigenomicFeature (feature_class, name) VALUES (%s, %s)",
                (feature_class, feature_name)
            )
            db.commit()
            feature_ids_cache_global[feature_name] = cursor.lastrowid

    feature_value_batch = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Inserting rows for {file}", leave=False):
        chrom = row['chrom']
        start = int(row['start'])
        end = int(row['end'])
        strand = row['strand']
        tpm = sanitize_value(row['TPM'])

        region_key = (chrom, start, end, strand)
        if region_key in region_cache:
            region_id = region_cache[region_key]
        else:
            cursor.execute(
                "SELECT region_id FROM GenomicRegion WHERE chrom = %s AND start = %s AND end = %s AND strand = %s",
                region_key
            )
            res = cursor.fetchone()
            if res:
                region_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO GenomicRegion (chrom, start, end, strand) VALUES (%s, %s, %s, %s)",
                    region_key
                )
                db.commit()
                region_id = cursor.lastrowid
            region_cache[region_key] = region_id

        activity_key = (region_id, cell_line_id, region_type_id, window_size_id)
        if activity_key in activity_cache:
            activity_id = activity_cache[activity_key]
        else:
            cursor.execute(
                "SELECT activity_id FROM EpigenomicActivity WHERE region_id = %s AND cell_line_id = %s AND region_type_id = %s AND window_size_id = %s",
                activity_key
            )
            res = cursor.fetchone()
            if res:
                activity_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO EpigenomicActivity (region_id, cell_line_id, region_type_id, window_size_id, TPM) VALUES (%s, %s, %s, %s, %s)",
                    (*activity_key, tpm)
                )
                db.commit()
                activity_id = cursor.lastrowid
            activity_cache[activity_key] = activity_id

        for feature_name in feature_cols:
            fold_change = sanitize_value(row[feature_name])
            if fold_change is None:
                continue
            feature_id = feature_ids_cache_global[feature_name]
            feature_value_batch.append((activity_id, feature_id, fold_change))

        if len(feature_value_batch) >= 1000:
            cursor.executemany(
                "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
                feature_value_batch
            )
            db.commit()
            feature_value_batch = []

    if feature_value_batch:
        cursor.executemany(
            "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
            feature_value_batch
        )
        db.commit()

    with open(processed_files_path, "a") as f:
        f.write(file + "\n")

    print(f"✅ Finished processing: {file}")

cursor.close()
db.close()
print("✅ All files processed.")

Processing CSV files:  11%|█         | 1/9 [5:21:51<42:54:52, 19311.52s/it]


✅ Finished processing: H1_1024_promoter_epigenomic.csv


FileNotFoundError: [Errno 2] No such file or directory: 'E:/DataTesis/Final_Epigenomic\\HEK293_1024_enhancer_epigenomic.csv'

In [2]:
import os
import math
import pandas as pd
import mysql.connector
from tqdm import tqdm

def sanitize_value(val):
    if val is None:
        return None
    if isinstance(val, float) and math.isnan(val):
        return None
    return val

# Feature class map
feature_class_map = {
    "Histone Modification": [
        "H3K27ac", "H3K27me3", "H3K36me3", "H3K4me1", "H3K4me2", "H3K4me3",
        "H3K79me2", "H3K9ac", "H3K9me2", "H3K9me3", "H4K20me1"
    ],
    "DNA Methylation": ["DNMT1", "DNMT3B"],
    "Chromatin Remodeling and Histone Modifiers": [
        "EZH2", "EED", "SUZ12", "KDM1A", "KDM2A", "KDM3A", "KDM4B", "KDM5A",
        "KDM5B", "KDM6A", "KAT2A", "KAT2B", "KAT7", "KAT8", "HDAC1", "HDAC2",
        "HDAC3", "HDAC6", "HDAC8", "BRD4"
    ],
    "Chromatin Structures and Remodelling Factors": [
        "CHD1", "CHD2", "CHD4", "CHD7", "ARID1B", "ARID2", "ARID3A", "ARID3B",
        "ARID4A", "ARID4B", "ARID5B", "SMARCA4", "SMARCB1"
    ],
    "Transcription Factor with Epigenetic Roles": [
        "CTCF", "CTCFL", "FOXA1", "FOXA2", "FOXA3", "GATA1", "GATA2", "GATA3",
        "NR3C1", "ESR1", "ETS1", "ETS2"
    ]
}
feature_to_class = {f: cls for cls, features in feature_class_map.items() for f in features}

# MySQL connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="B@ndung40175",
    database="epidb"
)
cursor = db.cursor()

csv_folder = "E:/DataTesis/Final_Epigenomic"
processed_files_path = "processed_files.txt"
already_processed = set()
if os.path.exists(processed_files_path):
    with open(processed_files_path, "r") as f:
        already_processed = set(f.read().splitlines())

csv_files = [
    f for f in os.listdir(csv_folder)
    if f.endswith(".csv") and f not in already_processed
]

# Caches
cell_line_cache = {}
region_type_cache = {}
window_size_cache = {}
feature_ids_cache_global = {}
region_cache = {}
activity_cache = {}

# Correct ID column names
id_column_map = {
    "CellLine": "cell_line_id",
    "RegionType": "region_type_id",
    "WindowSize": "window_size_id"
}

def get_or_insert_id(table, column, value, cache):
    if value in cache:
        return cache[value]

    id_column = id_column_map[table]
    cursor.execute(f"SELECT {id_column} FROM {table} WHERE {column} = %s", (value,))
    res = cursor.fetchone()
    
    if res:
        cache[value] = res[0]
        return res[0]

    cursor.execute(f"INSERT INTO {table} ({column}) VALUES (%s)", (value,))
    db.commit()  # ✅ Fix: use the correct connection
    new_id = cursor.lastrowid
    cache[value] = new_id
    return new_id

for file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(file_path)

    base_name = os.path.basename(file).replace("_epigenomic.csv", "")
    cell_line, window_size, region_type = base_name.split("_")
    window_size = int(window_size)

    cell_line_id = get_or_insert_id("CellLine", "name", cell_line, cell_line_cache)
    region_type_id = get_or_insert_id("RegionType", "type", region_type, region_type_cache)
    window_size_id = get_or_insert_id("WindowSize", "size", window_size, window_size_cache)

    fixed_cols = {"chrom", "start", "end", "strand", "TPM"}
    feature_cols = [col for col in df.columns if col not in fixed_cols]

    for feature_name in feature_cols:
        if feature_name in feature_ids_cache_global:
            continue
        feature_class = feature_to_class.get(feature_name, "Unknown")
        cursor.execute(
            "SELECT feature_id FROM EpigenomicFeature WHERE name = %s AND feature_class = %s",
            (feature_name, feature_class)
        )
        res = cursor.fetchone()
        if res:
            feature_ids_cache_global[feature_name] = res[0]
        else:
            cursor.execute(
                "INSERT INTO EpigenomicFeature (feature_class, name) VALUES (%s, %s)",
                (feature_class, feature_name)
            )
            db.commit()
            feature_ids_cache_global[feature_name] = cursor.lastrowid

    feature_value_batch = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Inserting rows for {file}", leave=False):
        chrom = row['chrom']
        start = int(row['start'])
        end = int(row['end'])
        strand = row['strand']
        tpm = sanitize_value(row['TPM'])

        region_key = (chrom, start, end, strand)
        if region_key in region_cache:
            region_id = region_cache[region_key]
        else:
            cursor.execute(
                "SELECT region_id FROM GenomicRegion WHERE chrom = %s AND start = %s AND end = %s AND strand = %s",
                region_key
            )
            res = cursor.fetchone()
            if res:
                region_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO GenomicRegion (chrom, start, end, strand) VALUES (%s, %s, %s, %s)",
                    region_key
                )
                db.commit()
                region_id = cursor.lastrowid
            region_cache[region_key] = region_id

        activity_key = (region_id, cell_line_id, region_type_id, window_size_id)
        if activity_key in activity_cache:
            activity_id = activity_cache[activity_key]
        else:
            cursor.execute(
                "SELECT activity_id FROM EpigenomicActivity WHERE region_id = %s AND cell_line_id = %s AND region_type_id = %s AND window_size_id = %s",
                activity_key
            )
            res = cursor.fetchone()
            if res:
                activity_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO EpigenomicActivity (region_id, cell_line_id, region_type_id, window_size_id, TPM) VALUES (%s, %s, %s, %s, %s)",
                    (*activity_key, tpm)
                )
                db.commit()
                activity_id = cursor.lastrowid
            activity_cache[activity_key] = activity_id

        for feature_name in feature_cols:
            fold_change = sanitize_value(row[feature_name])
            if fold_change is None:
                continue
            feature_id = feature_ids_cache_global[feature_name]
            feature_value_batch.append((activity_id, feature_id, fold_change))

        if len(feature_value_batch) >= 100000:
            cursor.executemany(
                "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
                feature_value_batch
            )
            db.commit()
            feature_value_batch = []

    if feature_value_batch:
        cursor.executemany(
            "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
            feature_value_batch
        )
        db.commit()

    with open(processed_files_path, "a") as f:
        f.write(file + "\n")

    print(f"✅ Finished processing: {file}")

cursor.close()
db.close()
print("✅ All files processed.")

Processing CSV files:  12%|█▎        | 1/8 [3:36:59<25:18:58, 13019.75s/it]

✅ Finished processing: HEK293_1024_enhancer_epigenomic.csv


Processing CSV files:  25%|██▌       | 2/8 [8:56:56<27:45:20, 16653.38s/it]

✅ Finished processing: HEK293_1024_promoter_epigenomic.csv


Processing CSV files:  38%|███▊      | 3/8 [9:00:23<12:41:58, 9143.74s/it] 

...

✅ Finished processing: K562_1024_promoter_epigenomic.csv


Processing CSV files:  88%|████████▊ | 7/8 [9:18:05<29:39, 1779.80s/it]  

✅ Finished processing: MCF-7_1024_enhancer_epigenomic.csv


Processing CSV files: 100%|██████████| 8/8 [9:22:55<00:00, 4221.94s/it]

✅ Finished processing: MCF-7_1024_promoter_epigenomic.csv
✅ All files processed.


In [3]:
import os
import math
import pandas as pd
import mysql.connector
from tqdm import tqdm

def sanitize_value(val):
    if val is None:
        return None
    if isinstance(val, float) and math.isnan(val):
        return None
    return val

# Feature class map
feature_class_map = {
    "Histone Modification": [
        "H3K27ac", "H3K27me3", "H3K36me3", "H3K4me1", "H3K4me2", "H3K4me3",
        "H3K79me2", "H3K9ac", "H3K9me2", "H3K9me3", "H4K20me1"
    ],
    "DNA Methylation": ["DNMT1", "DNMT3B"],
    "Chromatin Remodeling and Histone Modifiers": [
        "EZH2", "EED", "SUZ12", "KDM1A", "KDM2A", "KDM3A", "KDM4B", "KDM5A",
        "KDM5B", "KDM6A", "KAT2A", "KAT2B", "KAT7", "KAT8", "HDAC1", "HDAC2",
        "HDAC3", "HDAC6", "HDAC8", "BRD4"
    ],
    "Chromatin Structures and Remodelling Factors": [
        "CHD1", "CHD2", "CHD4", "CHD7", "ARID1B", "ARID2", "ARID3A", "ARID3B",
        "ARID4A", "ARID4B", "ARID5B", "SMARCA4", "SMARCB1"
    ],
    "Transcription Factor with Epigenetic Roles": [
        "CTCF", "CTCFL", "FOXA1", "FOXA2", "FOXA3", "GATA1", "GATA2", "GATA3",
        "NR3C1", "ESR1", "ETS1", "ETS2"
    ]
}
feature_to_class = {f: cls for cls, features in feature_class_map.items() for f in features}

# MySQL connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="B@ndung40175",
    database="epidb"
)
cursor = db.cursor()

csv_folder = "E:/DataTesis/Final_Epigenomic"
processed_files_path = "processed_files.txt"
already_processed = set()
if os.path.exists(processed_files_path):
    with open(processed_files_path, "r") as f:
        already_processed = set(f.read().splitlines())

csv_files = [
    f for f in os.listdir(csv_folder)
    if f.endswith(".csv") and f not in already_processed
]

# Caches
cell_line_cache = {}
region_type_cache = {}
window_size_cache = {}
feature_ids_cache_global = {}
region_cache = {}
activity_cache = {}

# Correct ID column names
id_column_map = {
    "CellLine": "cell_line_id",
    "RegionType": "region_type_id",
    "WindowSize": "window_size_id"
}

def get_or_insert_id(table, column, value, cache):
    if value in cache:
        return cache[value]

    id_column = id_column_map[table]
    cursor.execute(f"SELECT {id_column} FROM {table} WHERE {column} = %s", (value,))
    res = cursor.fetchone()
    
    if res:
        cache[value] = res[0]
        return res[0]

    cursor.execute(f"INSERT INTO {table} ({column}) VALUES (%s)", (value,))
    db.commit()  # ✅ Fix: use the correct connection
    new_id = cursor.lastrowid
    cache[value] = new_id
    return new_id

for file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(file_path)

    base_name = os.path.basename(file).replace("_epigenomic.csv", "")
    cell_line, window_size, region_type = base_name.split("_")
    window_size = int(window_size)

    cell_line_id = get_or_insert_id("CellLine", "name", cell_line, cell_line_cache)
    region_type_id = get_or_insert_id("RegionType", "type", region_type, region_type_cache)
    window_size_id = get_or_insert_id("WindowSize", "size", window_size, window_size_cache)

    fixed_cols = {"chrom", "start", "end", "strand", "TPM"}
    feature_cols = [col for col in df.columns if col not in fixed_cols]

    for feature_name in feature_cols:
        if feature_name in feature_ids_cache_global:
            continue
        feature_class = feature_to_class.get(feature_name, "Unknown")
        cursor.execute(
            "SELECT feature_id FROM EpigenomicFeature WHERE name = %s AND feature_class = %s",
            (feature_name, feature_class)
        )
        res = cursor.fetchone()
        if res:
            feature_ids_cache_global[feature_name] = res[0]
        else:
            cursor.execute(
                "INSERT INTO EpigenomicFeature (feature_class, name) VALUES (%s, %s)",
                (feature_class, feature_name)
            )
            db.commit()
            feature_ids_cache_global[feature_name] = cursor.lastrowid

    feature_value_batch = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Inserting rows for {file}", leave=False):
        chrom = row['chrom']
        start = int(row['start'])
        end = int(row['end'])
        strand = row['strand']
        tpm = sanitize_value(row['TPM'])

        region_key = (chrom, start, end, strand)
        if region_key in region_cache:
            region_id = region_cache[region_key]
        else:
            cursor.execute(
                "SELECT region_id FROM GenomicRegion WHERE chrom = %s AND start = %s AND end = %s AND strand = %s",
                region_key
            )
            res = cursor.fetchone()
            if res:
                region_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO GenomicRegion (chrom, start, end, strand) VALUES (%s, %s, %s, %s)",
                    region_key
                )
                db.commit()
                region_id = cursor.lastrowid
            region_cache[region_key] = region_id

        activity_key = (region_id, cell_line_id, region_type_id, window_size_id)
        if activity_key in activity_cache:
            activity_id = activity_cache[activity_key]
        else:
            cursor.execute(
                "SELECT activity_id FROM EpigenomicActivity WHERE region_id = %s AND cell_line_id = %s AND region_type_id = %s AND window_size_id = %s",
                activity_key
            )
            res = cursor.fetchone()
            if res:
                activity_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO EpigenomicActivity (region_id, cell_line_id, region_type_id, window_size_id, TPM) VALUES (%s, %s, %s, %s, %s)",
                    (*activity_key, tpm)
                )
                db.commit()
                activity_id = cursor.lastrowid
            activity_cache[activity_key] = activity_id

        for feature_name in feature_cols:
            fold_change = sanitize_value(row[feature_name])
            if fold_change is None:
                continue
            feature_id = feature_ids_cache_global[feature_name]
            feature_value_batch.append((activity_id, feature_id, fold_change))

        if len(feature_value_batch) >= 100000:
            cursor.executemany(
                "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
                feature_value_batch
            )
            db.commit()
            feature_value_batch = []

    if feature_value_batch:
        cursor.executemany(
            "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
            feature_value_batch
        )
        db.commit()

    with open(processed_files_path, "a") as f:
        f.write(file + "\n")

    print(f"✅ Finished processing: {file}")

cursor.close()
db.close()
print("✅ All files processed.")

Processing CSV files:   2%|▏         | 1/48 [3:54:45<183:54:01, 14085.99s/it]

✅ Finished processing: GM12878_128_enhancer_epigenomic.csv


Processing CSV files:   4%|▍         | 2/48 [9:35:15<227:40:02, 17817.45s/it]

✅ Finished processing: GM12878_128_promoter_epigenomic.csv


Processing CSV files:   6%|▋         | 3/48 [13:16:43<196:52:01, 15749.38s/it]

✅ Finished processing: GM12878_256_enhancer_epigenomic.csv


Processing CSV files:   8%|▊         | 4/48 [18:40:48<205:28:53, 16812.12s/it]

✅ Finished processing: GM12878_256_promoter_epigenomic.csv


FileNotFoundError: [Errno 2] No such file or directory: 'E:/DataTesis/Final_Epigenomic\\GM12878_512_enhancer_epigenomic.csv'

In [4]:
import os
import math
import pandas as pd
import mysql.connector
from tqdm import tqdm

def sanitize_value(val):
    if val is None:
        return None
    if isinstance(val, float) and math.isnan(val):
        return None
    return val

# Feature class map
feature_class_map = {
    "Histone Modification": [
        "H3K27ac", "H3K27me3", "H3K36me3", "H3K4me1", "H3K4me2", "H3K4me3",
        "H3K79me2", "H3K9ac", "H3K9me2", "H3K9me3", "H4K20me1"
    ],
    "DNA Methylation": ["DNMT1", "DNMT3B"],
    "Chromatin Remodeling and Histone Modifiers": [
        "EZH2", "EED", "SUZ12", "KDM1A", "KDM2A", "KDM3A", "KDM4B", "KDM5A",
        "KDM5B", "KDM6A", "KAT2A", "KAT2B", "KAT7", "KAT8", "HDAC1", "HDAC2",
        "HDAC3", "HDAC6", "HDAC8", "BRD4"
    ],
    "Chromatin Structures and Remodelling Factors": [
        "CHD1", "CHD2", "CHD4", "CHD7", "ARID1B", "ARID2", "ARID3A", "ARID3B",
        "ARID4A", "ARID4B", "ARID5B", "SMARCA4", "SMARCB1"
    ],
    "Transcription Factor with Epigenetic Roles": [
        "CTCF", "CTCFL", "FOXA1", "FOXA2", "FOXA3", "GATA1", "GATA2", "GATA3",
        "NR3C1", "ESR1", "ETS1", "ETS2"
    ]
}
feature_to_class = {f: cls for cls, features in feature_class_map.items() for f in features}

# MySQL connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="B@ndung40175",
    database="epidb"
)
cursor = db.cursor()

csv_folder = "E:/DataTesis/Final_Epigenomic"
processed_files_path = "processed_files.txt"
already_processed = set()
if os.path.exists(processed_files_path):
    with open(processed_files_path, "r") as f:
        already_processed = set(f.read().splitlines())

csv_files = [
    f for f in os.listdir(csv_folder)
    if f.endswith(".csv") and f not in already_processed
]

# Caches
cell_line_cache = {}
region_type_cache = {}
window_size_cache = {}
feature_ids_cache_global = {}
region_cache = {}
activity_cache = {}

# Correct ID column names
id_column_map = {
    "CellLine": "cell_line_id",
    "RegionType": "region_type_id",
    "WindowSize": "window_size_id"
}

def get_or_insert_id(table, column, value, cache):
    if value in cache:
        return cache[value]

    id_column = id_column_map[table]
    cursor.execute(f"SELECT {id_column} FROM {table} WHERE {column} = %s", (value,))
    res = cursor.fetchone()
    
    if res:
        cache[value] = res[0]
        return res[0]

    cursor.execute(f"INSERT INTO {table} ({column}) VALUES (%s)", (value,))
    db.commit()  # ✅ Fix: use the correct connection
    new_id = cursor.lastrowid
    cache[value] = new_id
    return new_id

for file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(file_path)

    base_name = os.path.basename(file).replace("_epigenomic.csv", "")
    cell_line, window_size, region_type = base_name.split("_")
    window_size = int(window_size)

    cell_line_id = get_or_insert_id("CellLine", "name", cell_line, cell_line_cache)
    region_type_id = get_or_insert_id("RegionType", "type", region_type, region_type_cache)
    window_size_id = get_or_insert_id("WindowSize", "size", window_size, window_size_cache)

    fixed_cols = {"chrom", "start", "end", "strand", "TPM"}
    feature_cols = [col for col in df.columns if col not in fixed_cols]

    for feature_name in feature_cols:
        if feature_name in feature_ids_cache_global:
            continue
        feature_class = feature_to_class.get(feature_name, "Unknown")
        cursor.execute(
            "SELECT feature_id FROM EpigenomicFeature WHERE name = %s AND feature_class = %s",
            (feature_name, feature_class)
        )
        res = cursor.fetchone()
        if res:
            feature_ids_cache_global[feature_name] = res[0]
        else:
            cursor.execute(
                "INSERT INTO EpigenomicFeature (feature_class, name) VALUES (%s, %s)",
                (feature_class, feature_name)
            )
            db.commit()
            feature_ids_cache_global[feature_name] = cursor.lastrowid

    feature_value_batch = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Inserting rows for {file}", leave=False):
        chrom = row['chrom']
        start = int(row['start'])
        end = int(row['end'])
        strand = row['strand']
        tpm = sanitize_value(row['TPM'])

        region_key = (chrom, start, end, strand)
        if region_key in region_cache:
            region_id = region_cache[region_key]
        else:
            cursor.execute(
                "SELECT region_id FROM GenomicRegion WHERE chrom = %s AND start = %s AND end = %s AND strand = %s",
                region_key
            )
            res = cursor.fetchone()
            if res:
                region_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO GenomicRegion (chrom, start, end, strand) VALUES (%s, %s, %s, %s)",
                    region_key
                )
                db.commit()
                region_id = cursor.lastrowid
            region_cache[region_key] = region_id

        activity_key = (region_id, cell_line_id, region_type_id, window_size_id)
        if activity_key in activity_cache:
            activity_id = activity_cache[activity_key]
        else:
            cursor.execute(
                "SELECT activity_id FROM EpigenomicActivity WHERE region_id = %s AND cell_line_id = %s AND region_type_id = %s AND window_size_id = %s",
                activity_key
            )
            res = cursor.fetchone()
            if res:
                activity_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO EpigenomicActivity (region_id, cell_line_id, region_type_id, window_size_id, TPM) VALUES (%s, %s, %s, %s, %s)",
                    (*activity_key, tpm)
                )
                db.commit()
                activity_id = cursor.lastrowid
            activity_cache[activity_key] = activity_id

        for feature_name in feature_cols:
            fold_change = sanitize_value(row[feature_name])
            if fold_change is None:
                continue
            feature_id = feature_ids_cache_global[feature_name]
            feature_value_batch.append((activity_id, feature_id, fold_change))

        if len(feature_value_batch) >= 100000:
            cursor.executemany(
                "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
                feature_value_batch
            )
            db.commit()
            feature_value_batch = []

    if feature_value_batch:
        cursor.executemany(
            "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
            feature_value_batch
        )
        db.commit()

    with open(processed_files_path, "a") as f:
        f.write(file + "\n")

    print(f"✅ Finished processing: {file}")

cursor.close()
db.close()
print("✅ All files processed.")

Processing CSV files:  10%|█         | 1/10 [3:20:30<30:04:34, 12030.53s/it]

✅ Finished processing: H1_128_enhancer_epigenomic.csv


Processing CSV files:  20%|██        | 2/10 [8:41:27<36:10:50, 16281.27s/it]

✅ Finished processing: H1_128_promoter_epigenomic.csv


Processing CSV files:  30%|███       | 3/10 [8:44:01<17:20:21, 8917.35s/it] 

...

✅ Finished processing: K562_128_promoter_epigenomic.csv


Processing CSV files:  90%|█████████ | 9/10 [9:08:00<15:31, 931.11s/it] 

✅ Finished processing: MCF-7_128_enhancer_epigenomic.csv


Processing CSV files: 100%|██████████| 10/10 [9:12:34<00:00, 3315.41s/it]

✅ Finished processing: MCF-7_128_promoter_epigenomic.csv
✅ All files processed.


In [5]:
import os
import math
import pandas as pd
import mysql.connector
from tqdm import tqdm

def sanitize_value(val):
    if val is None:
        return None
    if isinstance(val, float) and math.isnan(val):
        return None
    return val

# Feature class map
feature_class_map = {
    "Histone Modification": [
        "H3K27ac", "H3K27me3", "H3K36me3", "H3K4me1", "H3K4me2", "H3K4me3",
        "H3K79me2", "H3K9ac", "H3K9me2", "H3K9me3", "H4K20me1"
    ],
    "DNA Methylation": ["DNMT1", "DNMT3B"],
    "Chromatin Remodeling and Histone Modifiers": [
        "EZH2", "EED", "SUZ12", "KDM1A", "KDM2A", "KDM3A", "KDM4B", "KDM5A",
        "KDM5B", "KDM6A", "KAT2A", "KAT2B", "KAT7", "KAT8", "HDAC1", "HDAC2",
        "HDAC3", "HDAC6", "HDAC8", "BRD4"
    ],
    "Chromatin Structures and Remodelling Factors": [
        "CHD1", "CHD2", "CHD4", "CHD7", "ARID1B", "ARID2", "ARID3A", "ARID3B",
        "ARID4A", "ARID4B", "ARID5B", "SMARCA4", "SMARCB1"
    ],
    "Transcription Factor with Epigenetic Roles": [
        "CTCF", "CTCFL", "FOXA1", "FOXA2", "FOXA3", "GATA1", "GATA2", "GATA3",
        "NR3C1", "ESR1", "ETS1", "ETS2"
    ]
}
feature_to_class = {f: cls for cls, features in feature_class_map.items() for f in features}

# MySQL connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="B@ndung40175",
    database="epidb"
)
cursor = db.cursor()

csv_folder = "E:/DataTesis/Final_Epigenomic"
processed_files_path = "processed_files.txt"
already_processed = set()
if os.path.exists(processed_files_path):
    with open(processed_files_path, "r") as f:
        already_processed = set(f.read().splitlines())

csv_files = [
    f for f in os.listdir(csv_folder)
    if f.endswith(".csv") and f not in already_processed
]

# Caches
cell_line_cache = {}
region_type_cache = {}
window_size_cache = {}
feature_ids_cache_global = {}
region_cache = {}
activity_cache = {}

# Correct ID column names
id_column_map = {
    "CellLine": "cell_line_id",
    "RegionType": "region_type_id",
    "WindowSize": "window_size_id"
}

def get_or_insert_id(table, column, value, cache):
    if value in cache:
        return cache[value]

    id_column = id_column_map[table]
    cursor.execute(f"SELECT {id_column} FROM {table} WHERE {column} = %s", (value,))
    res = cursor.fetchone()
    
    if res:
        cache[value] = res[0]
        return res[0]

    cursor.execute(f"INSERT INTO {table} ({column}) VALUES (%s)", (value,))
    db.commit()  # ✅ Fix: use the correct connection
    new_id = cursor.lastrowid
    cache[value] = new_id
    return new_id

for file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(file_path)

    base_name = os.path.basename(file).replace("_epigenomic.csv", "")
    cell_line, window_size, region_type = base_name.split("_")
    window_size = int(window_size)

    cell_line_id = get_or_insert_id("CellLine", "name", cell_line, cell_line_cache)
    region_type_id = get_or_insert_id("RegionType", "type", region_type, region_type_cache)
    window_size_id = get_or_insert_id("WindowSize", "size", window_size, window_size_cache)

    fixed_cols = {"chrom", "start", "end", "strand", "TPM"}
    feature_cols = [col for col in df.columns if col not in fixed_cols]

    for feature_name in feature_cols:
        if feature_name in feature_ids_cache_global:
            continue
        feature_class = feature_to_class.get(feature_name, "Unknown")
        cursor.execute(
            "SELECT feature_id FROM EpigenomicFeature WHERE name = %s AND feature_class = %s",
            (feature_name, feature_class)
        )
        res = cursor.fetchone()
        if res:
            feature_ids_cache_global[feature_name] = res[0]
        else:
            cursor.execute(
                "INSERT INTO EpigenomicFeature (feature_class, name) VALUES (%s, %s)",
                (feature_class, feature_name)
            )
            db.commit()
            feature_ids_cache_global[feature_name] = cursor.lastrowid

    feature_value_batch = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Inserting rows for {file}", leave=False):
        chrom = row['chrom']
        start = int(row['start'])
        end = int(row['end'])
        strand = row['strand']
        tpm = sanitize_value(row['TPM'])

        region_key = (chrom, start, end, strand)
        if region_key in region_cache:
            region_id = region_cache[region_key]
        else:
            cursor.execute(
                "SELECT region_id FROM GenomicRegion WHERE chrom = %s AND start = %s AND end = %s AND strand = %s",
                region_key
            )
            res = cursor.fetchone()
            if res:
                region_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO GenomicRegion (chrom, start, end, strand) VALUES (%s, %s, %s, %s)",
                    region_key
                )
                db.commit()
                region_id = cursor.lastrowid
            region_cache[region_key] = region_id

        activity_key = (region_id, cell_line_id, region_type_id, window_size_id)
        if activity_key in activity_cache:
            activity_id = activity_cache[activity_key]
        else:
            cursor.execute(
                "SELECT activity_id FROM EpigenomicActivity WHERE region_id = %s AND cell_line_id = %s AND region_type_id = %s AND window_size_id = %s",
                activity_key
            )
            res = cursor.fetchone()
            if res:
                activity_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO EpigenomicActivity (region_id, cell_line_id, region_type_id, window_size_id, TPM) VALUES (%s, %s, %s, %s, %s)",
                    (*activity_key, tpm)
                )
                db.commit()
                activity_id = cursor.lastrowid
            activity_cache[activity_key] = activity_id

        for feature_name in feature_cols:
            fold_change = sanitize_value(row[feature_name])
            if fold_change is None:
                continue
            feature_id = feature_ids_cache_global[feature_name]
            feature_value_batch.append((activity_id, feature_id, fold_change))

        if len(feature_value_batch) >= 100000:
            cursor.executemany(
                "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
                feature_value_batch
            )
            db.commit()
            feature_value_batch = []

    if feature_value_batch:
        cursor.executemany(
            "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
            feature_value_batch
        )
        db.commit()

    with open(processed_files_path, "a") as f:
        f.write(file + "\n")

    print(f"✅ Finished processing: {file}")

cursor.close()
db.close()
print("✅ All files processed.")

Processing CSV files:   8%|▊         | 1/12 [3:28:51<38:17:26, 12531.47s/it]

✅ Finished processing: GM12878_64_enhancer_epigenomic.csv


Processing CSV files:  17%|█▋        | 2/12 [8:36:23<44:28:59, 16013.98s/it]

✅ Finished processing: GM12878_64_promoter_epigenomic.csv


Processing CSV files:  25%|██▌       | 3/12 [8:39:00<21:56:00, 8773.38s/it] 

...

✅ Finished processing: K562_64_promoter_epigenomic.csv


Processing CSV files:  92%|█████████▏| 11/12 [9:08:11<09:08, 548.01s/it]

✅ Finished processing: MCF-7_64_enhancer_epigenomic.csv


Processing CSV files: 100%|██████████| 12/12 [9:12:24<00:00, 2762.01s/it]

✅ Finished processing: MCF-7_64_promoter_epigenomic.csv
✅ All files processed.


In [1]:
import os
import math
import pandas as pd
import mysql.connector
from tqdm import tqdm

def sanitize_value(val):
    if val is None:
        return None
    if isinstance(val, float) and math.isnan(val):
        return None
    return val

# Feature class map
feature_class_map = {
    "Histone Modification": [
        "H3K27ac", "H3K27me3", "H3K36me3", "H3K4me1", "H3K4me2", "H3K4me3",
        "H3K79me2", "H3K9ac", "H3K9me2", "H3K9me3", "H4K20me1"
    ],
    "DNA Methylation": ["DNMT1", "DNMT3B"],
    "Chromatin Remodeling and Histone Modifiers": [
        "EZH2", "EED", "SUZ12", "KDM1A", "KDM2A", "KDM3A", "KDM4B", "KDM5A",
        "KDM5B", "KDM6A", "KAT2A", "KAT2B", "KAT7", "KAT8", "HDAC1", "HDAC2",
        "HDAC3", "HDAC6", "HDAC8", "BRD4"
    ],
    "Chromatin Structures and Remodelling Factors": [
        "CHD1", "CHD2", "CHD4", "CHD7", "ARID1B", "ARID2", "ARID3A", "ARID3B",
        "ARID4A", "ARID4B", "ARID5B", "SMARCA4", "SMARCB1"
    ],
    "Transcription Factor with Epigenetic Roles": [
        "CTCF", "CTCFL", "FOXA1", "FOXA2", "FOXA3", "GATA1", "GATA2", "GATA3",
        "NR3C1", "ESR1", "ETS1", "ETS2"
    ]
}
feature_to_class = {f: cls for cls, features in feature_class_map.items() for f in features}

# MySQL connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="B@ndung40175",
    database="epidb"
)
cursor = db.cursor()

csv_folder = "E:/DataTesis/Final_Epigenomic"
processed_files_path = "processed_files.txt"
already_processed = set()
if os.path.exists(processed_files_path):
    with open(processed_files_path, "r") as f:
        already_processed = set(f.read().splitlines())

csv_files = [
    f for f in os.listdir(csv_folder)
    if f.endswith(".csv") and f not in already_processed
]

# Caches
cell_line_cache = {}
region_type_cache = {}
window_size_cache = {}
feature_ids_cache_global = {}
region_cache = {}
activity_cache = {}

# Correct ID column names
id_column_map = {
    "CellLine": "cell_line_id",
    "RegionType": "region_type_id",
    "WindowSize": "window_size_id"
}

def get_or_insert_id(table, column, value, cache):
    if value in cache:
        return cache[value]

    id_column = id_column_map[table]
    cursor.execute(f"SELECT {id_column} FROM {table} WHERE {column} = %s", (value,))
    res = cursor.fetchone()
    
    if res:
        cache[value] = res[0]
        return res[0]

    cursor.execute(f"INSERT INTO {table} ({column}) VALUES (%s)", (value,))
    db.commit()  # ✅ Fix: use the correct connection
    new_id = cursor.lastrowid
    cache[value] = new_id
    return new_id

for file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(file_path)

    base_name = os.path.basename(file).replace("_epigenomic.csv", "")
    cell_line, window_size, region_type = base_name.split("_")
    window_size = int(window_size)

    cell_line_id = get_or_insert_id("CellLine", "name", cell_line, cell_line_cache)
    region_type_id = get_or_insert_id("RegionType", "type", region_type, region_type_cache)
    window_size_id = get_or_insert_id("WindowSize", "size", window_size, window_size_cache)

    fixed_cols = {"chrom", "start", "end", "strand", "TPM"}
    feature_cols = [col for col in df.columns if col not in fixed_cols]

    for feature_name in feature_cols:
        if feature_name in feature_ids_cache_global:
            continue
        feature_class = feature_to_class.get(feature_name, "Unknown")
        cursor.execute(
            "SELECT feature_id FROM EpigenomicFeature WHERE name = %s AND feature_class = %s",
            (feature_name, feature_class)
        )
        res = cursor.fetchone()
        if res:
            feature_ids_cache_global[feature_name] = res[0]
        else:
            cursor.execute(
                "INSERT INTO EpigenomicFeature (feature_class, name) VALUES (%s, %s)",
                (feature_class, feature_name)
            )
            db.commit()
            feature_ids_cache_global[feature_name] = cursor.lastrowid

    feature_value_batch = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Inserting rows for {file}", leave=False):
        chrom = row['chrom']
        start = int(row['start'])
        end = int(row['end'])
        strand = row['strand']
        tpm = sanitize_value(row['TPM'])

        region_key = (chrom, start, end, strand)
        if region_key in region_cache:
            region_id = region_cache[region_key]
        else:
            cursor.execute(
                "SELECT region_id FROM GenomicRegion WHERE chrom = %s AND start = %s AND end = %s AND strand = %s",
                region_key
            )
            res = cursor.fetchone()
            if res:
                region_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO GenomicRegion (chrom, start, end, strand) VALUES (%s, %s, %s, %s)",
                    region_key
                )
                db.commit()
                region_id = cursor.lastrowid
            region_cache[region_key] = region_id

        activity_key = (region_id, cell_line_id, region_type_id, window_size_id)
        if activity_key in activity_cache:
            activity_id = activity_cache[activity_key]
        else:
            cursor.execute(
                "SELECT activity_id FROM EpigenomicActivity WHERE region_id = %s AND cell_line_id = %s AND region_type_id = %s AND window_size_id = %s",
                activity_key
            )
            res = cursor.fetchone()
            if res:
                activity_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO EpigenomicActivity (region_id, cell_line_id, region_type_id, window_size_id, TPM) VALUES (%s, %s, %s, %s, %s)",
                    (*activity_key, tpm)
                )
                db.commit()
                activity_id = cursor.lastrowid
            activity_cache[activity_key] = activity_id

        for feature_name in feature_cols:
            fold_change = sanitize_value(row[feature_name])
            if fold_change is None:
                continue
            feature_id = feature_ids_cache_global[feature_name]
            feature_value_batch.append((activity_id, feature_id, fold_change))

        if len(feature_value_batch) >= 100000:
            cursor.executemany(
                "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
                feature_value_batch
            )
            db.commit()
            feature_value_batch = []

    if feature_value_batch:
        cursor.executemany(
            "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
            feature_value_batch
        )
        db.commit()

    with open(processed_files_path, "a") as f:
        f.write(file + "\n")

    print(f"✅ Finished processing: {file}")

cursor.close()
db.close()
print("✅ All files processed.")

Processing CSV files:   8%|▊         | 1/12 [3:41:52<40:40:32, 13312.04s/it]

✅ Finished processing: GM12878_512_enhancer_epigenomic.csv


Processing CSV files:  17%|█▋        | 2/12 [9:18:57<48:16:24, 17378.50s/it]

✅ Finished processing: GM12878_512_promoter_epigenomic.csv


Processing CSV files:  25%|██▌       | 3/12 [9:21:52<23:48:30, 9523.44s/it] 

...

✅ Finished processing: K562_512_promoter_epigenomic.csv


Processing CSV files:  92%|█████████▏| 11/12 [9:54:22<09:59, 599.13s/it]

✅ Finished processing: MCF-7_512_enhancer_epigenomic.csv


Processing CSV files: 100%|██████████| 12/12 [9:59:05<00:00, 2995.50s/it]

✅ Finished processing: MCF-7_512_promoter_epigenomic.csv
✅ All files processed.


In [2]:
import os
import math
import pandas as pd
import mysql.connector
from tqdm import tqdm

def sanitize_value(val):
    if val is None:
        return None
    if isinstance(val, float) and math.isnan(val):
        return None
    return val

# Feature class map
feature_class_map = {
    "Histone Modification": [
        "H3K27ac", "H3K27me3", "H3K36me3", "H3K4me1", "H3K4me2", "H3K4me3",
        "H3K79me2", "H3K9ac", "H3K9me2", "H3K9me3", "H4K20me1"
    ],
    "DNA Methylation": ["DNMT1", "DNMT3B"],
    "Chromatin Remodeling and Histone Modifiers": [
        "EZH2", "EED", "SUZ12", "KDM1A", "KDM2A", "KDM3A", "KDM4B", "KDM5A",
        "KDM5B", "KDM6A", "KAT2A", "KAT2B", "KAT7", "KAT8", "HDAC1", "HDAC2",
        "HDAC3", "HDAC6", "HDAC8", "BRD4"
    ],
    "Chromatin Structures and Remodelling Factors": [
        "CHD1", "CHD2", "CHD4", "CHD7", "ARID1B", "ARID2", "ARID3A", "ARID3B",
        "ARID4A", "ARID4B", "ARID5B", "SMARCA4", "SMARCB1"
    ],
    "Transcription Factor with Epigenetic Roles": [
        "CTCF", "CTCFL", "FOXA1", "FOXA2", "FOXA3", "GATA1", "GATA2", "GATA3",
        "NR3C1", "ESR1", "ETS1", "ETS2"
    ]
}
feature_to_class = {f: cls for cls, features in feature_class_map.items() for f in features}

# MySQL connection
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="B@ndung40175",
    database="epidb"
)
cursor = db.cursor()

csv_folder = "E:/DataTesis/Final_Epigenomic"
processed_files_path = "processed_files.txt"
already_processed = set()
if os.path.exists(processed_files_path):
    with open(processed_files_path, "r") as f:
        already_processed = set(f.read().splitlines())

csv_files = [
    f for f in os.listdir(csv_folder)
    if f.endswith(".csv") and f not in already_processed
]

# Caches
cell_line_cache = {}
region_type_cache = {}
window_size_cache = {}
feature_ids_cache_global = {}
region_cache = {}
activity_cache = {}

# Correct ID column names
id_column_map = {
    "CellLine": "cell_line_id",
    "RegionType": "region_type_id",
    "WindowSize": "window_size_id"
}

def get_or_insert_id(table, column, value, cache):
    if value in cache:
        return cache[value]

    id_column = id_column_map[table]
    cursor.execute(f"SELECT {id_column} FROM {table} WHERE {column} = %s", (value,))
    res = cursor.fetchone()
    
    if res:
        cache[value] = res[0]
        return res[0]

    cursor.execute(f"INSERT INTO {table} ({column}) VALUES (%s)", (value,))
    db.commit()  # ✅ Fix: use the correct connection
    new_id = cursor.lastrowid
    cache[value] = new_id
    return new_id

for file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(file_path)

    base_name = os.path.basename(file).replace("_epigenomic.csv", "")
    cell_line, window_size, region_type = base_name.split("_")
    window_size = int(window_size)

    cell_line_id = get_or_insert_id("CellLine", "name", cell_line, cell_line_cache)
    region_type_id = get_or_insert_id("RegionType", "type", region_type, region_type_cache)
    window_size_id = get_or_insert_id("WindowSize", "size", window_size, window_size_cache)

    fixed_cols = {"chrom", "start", "end", "strand", "TPM"}
    feature_cols = [col for col in df.columns if col not in fixed_cols]

    for feature_name in feature_cols:
        if feature_name in feature_ids_cache_global:
            continue
        feature_class = feature_to_class.get(feature_name, "Unknown")
        cursor.execute(
            "SELECT feature_id FROM EpigenomicFeature WHERE name = %s AND feature_class = %s",
            (feature_name, feature_class)
        )
        res = cursor.fetchone()
        if res:
            feature_ids_cache_global[feature_name] = res[0]
        else:
            cursor.execute(
                "INSERT INTO EpigenomicFeature (feature_class, name) VALUES (%s, %s)",
                (feature_class, feature_name)
            )
            db.commit()
            feature_ids_cache_global[feature_name] = cursor.lastrowid

    feature_value_batch = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Inserting rows for {file}", leave=False):
        chrom = row['chrom']
        start = int(row['start'])
        end = int(row['end'])
        strand = row['strand']
        tpm = sanitize_value(row['TPM'])

        region_key = (chrom, start, end, strand)
        if region_key in region_cache:
            region_id = region_cache[region_key]
        else:
            cursor.execute(
                "SELECT region_id FROM GenomicRegion WHERE chrom = %s AND start = %s AND end = %s AND strand = %s",
                region_key
            )
            res = cursor.fetchone()
            if res:
                region_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO GenomicRegion (chrom, start, end, strand) VALUES (%s, %s, %s, %s)",
                    region_key
                )
                db.commit()
                region_id = cursor.lastrowid
            region_cache[region_key] = region_id

        activity_key = (region_id, cell_line_id, region_type_id, window_size_id)
        if activity_key in activity_cache:
            activity_id = activity_cache[activity_key]
        else:
            cursor.execute(
                "SELECT activity_id FROM EpigenomicActivity WHERE region_id = %s AND cell_line_id = %s AND region_type_id = %s AND window_size_id = %s",
                activity_key
            )
            res = cursor.fetchone()
            if res:
                activity_id = res[0]
            else:
                cursor.execute(
                    "INSERT INTO EpigenomicActivity (region_id, cell_line_id, region_type_id, window_size_id, TPM) VALUES (%s, %s, %s, %s, %s)",
                    (*activity_key, tpm)
                )
                db.commit()
                activity_id = cursor.lastrowid
            activity_cache[activity_key] = activity_id

        for feature_name in feature_cols:
            fold_change = sanitize_value(row[feature_name])
            if fold_change is None:
                continue
            feature_id = feature_ids_cache_global[feature_name]
            feature_value_batch.append((activity_id, feature_id, fold_change))

        if len(feature_value_batch) >= 100000:
            cursor.executemany(
                "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
                feature_value_batch
            )
            db.commit()
            feature_value_batch = []

    if feature_value_batch:
        cursor.executemany(
            "INSERT INTO FeatureValue (activity_id, feature_id, fold_change) VALUES (%s, %s, %s)",
            feature_value_batch
        )
        db.commit()

    with open(processed_files_path, "a") as f:
        f.write(file + "\n")

    print(f"✅ Finished processing: {file}")

cursor.close()
db.close()
print("✅ All files processed.")

Processing CSV files:  10%|█         | 1/10 [3:41:30<33:13:38, 13290.96s/it]

✅ Finished processing: H1_256_enhancer_epigenomic.csv


Processing CSV files:  20%|██        | 2/10 [8:47:53<36:11:27, 16285.95s/it]

✅ Finished processing: H1_256_promoter_epigenomic.csv


Processing CSV files:  30%|███       | 3/10 [8:50:18<17:20:08, 8915.52s/it] 

...

✅ Finished processing: K562_256_promoter_epigenomic.csv


Processing CSV files:  90%|█████████ | 9/10 [9:12:36<15:17, 917.32s/it] 

✅ Finished processing: MCF-7_256_enhancer_epigenomic.csv


Processing CSV files: 100%|██████████| 10/10 [9:16:50<00:00, 3341.00s/it]

✅ Finished processing: MCF-7_256_promoter_epigenomic.csv
✅ All files processed.
